In [2]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import folium
from scipy.spatial import distance
from geopy.distance import geodesic
from math import radians, sin, cos, atan2, sqrt
import branca

# Carga de datos de contaminantes del mes de Enero de 2024

Cargamos tanto los datos de contaminantes en diferentes estaciones como información de los diferentes puntos de medición

In [3]:
contaminante = 8

In [4]:
contaminacion_mes_madrid = pd.read_csv('https://www.dropbox.com/scl/fi/jciybhaty3mps5kfikra6/ene_mo24.csv?rlkey=jgelc5op7v0hbuosfk7lei956&st=0rukefww&dl=1',sep=';')
estaciones_df = pd.read_excel('https://www.dropbox.com/scl/fi/3mfaz79uow6pqud3j19vc/informacion_estaciones_red_calidad_aire.xls?rlkey=h1uaefe0mt87u7d0yvdneidg3&st=ihyekq04&dl=1')

In [5]:
estaciones_df

,CODIGO,CODIGO_CORTO,ESTACION,DIRECCION,LONGITUD_ETRS89,LATITUD_ETRS89,ALTITUD,COD_TIPO,NOM_TIPO,NO2,...,BTX,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,Fecha alta,COORDENADA_X_ETRS89,COORDENADA_Y_ETRS89,LONGITUD,LATITUD
0,28079004,4,Plaza de España,Plaza de España,"3°42'43.91""O","40°25'25.98""N",637,UT,Urbana tráfico,X,...,NaN,273600,PLAZA,DE,ESPAÑA,1998-12-01,439579.329073,4.475049e+06,-3.712257,40.423882
1,28079008,8,Escuelas Aguirre,Entre C/ Alcalá y C/ O’ Donell,"3°40'56.22""O","40°25'17.63""N",672,UT,Urbana tráfico,X,...,X,18900,CALLE,DE,ALCALA,1998-12-01,442117.236578,4.474771e+06,-3.682316,40.421553
2,28079011,11,Ramón y Cajal,Avda. Ramón y Cajal esq. C/ Príncipe de Vergara,"3°40'38.50""O","40°27'5.29""N",709,UT,Urbana tráfico,X,...,X,610450,CALLE,DEL,PRINCIPE DE VERGARA,1998-12-01,442564.045713,4.478089e+06,-3.677349,40.451473
3,28079016,16,Arturo Soria,C/ Arturo Soria esq. C/ Vizconde de los Asilos,"3°38'21.17""O","40°26'24.20""N",695,UF,Urbana fondo,X,...,NaN,798700,CALLE,DEL,VIZCONDE DE LOS ASILOS,1998-12-01,445786.172865,4.476796e+06,-3.639242,40.440046
4,28079017,17,Villaverde,C/ Juan Peñalver,"3°42'47.89""O","40°20'49.74""N",601,UF,Urbana fondo,X,...,NaN,417200,CALLE,DE,JUAN PEÑALVER,1998-12-01,439420.701532,4.466532e+06,-3.713317,40.347147
5,28079018,18,Farolillo,C/ Farolillo - C/ Ervigio,"3°43'54.61""O","40°23'41.22""N",625,UF,Urbana fondo,X,...,X,1903,CALLE,DEL,FAROLILLO,2001-03-31,437891.696120,4.471833e+06,-3.731836,40.394782
6,28079024,24,Casa de Campo,Casa de Campo (Terminal del Teleférico),"3°44'50.44""O","40°25'9.69""N",645,S,Suburbana,X,...,NaN,905219,CARRETERA,DEL,TELEFERICO,1998-12-01,436598.563744,4.474572e+06,-3.747345,40.419358
7,28079027,27,Barajas Pueblo,"C/ Júpiter, 21","3°34'48.10""O","40°28'36.93""N",619,UF,Urbana fondo,X,...,NaN,425700,CALLE,DE,JUPITER,2002-12-31,450835.202581,4.480854e+06,-3.580026,40.476918
8,28079035,35,Plaza del Carmen,Plaza del Carmen esq. Tres Cruces,"3°42'11.40""O","40°25'9.15""N",660,UF,Urbana fondo,X,...,NaN,145800,PLAZA,DEL,CARMEN,1999-11-01,440346.361892,4.474524e+06,-3.703166,40.419209
9,28079036,36,Moratalaz,Avda. Moratalaz esq. Camino de los Vinateros,"3°38'43.02""O","40°24'28.64""N",671,UT,Urbana tráfico,X,...,NaN,522000,AVENIDA,DE,MORATALAZ,1998-12-01,445245.513003,4.473237e+06,-3.645310,40.407952


Nos quedamos con las mediciones de NO2  

In [6]:
contaminacion_mes_madrid = contaminacion_mes_madrid[contaminacion_mes_madrid['MAGNITUD'] == contaminante]
contaminacion_mes_madrid

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
62,28,79,4,8,28079004_8_8,2024,1,1,15.0,V,...,63.0,V,58.0,V,61.0,V,59.0,V,55.0,V
63,28,79,4,8,28079004_8_8,2024,1,2,41.0,V,...,60.0,V,53.0,V,49.0,V,48.0,V,54.0,V
64,28,79,4,8,28079004_8_8,2024,1,3,45.0,V,...,38.0,V,43.0,V,43.0,V,39.0,V,51.0,V
65,28,79,4,8,28079004_8_8,2024,1,4,51.0,V,...,51.0,V,48.0,V,36.0,V,25.0,V,20.0,V
66,28,79,4,8,28079004_8_8,2024,1,5,19.0,V,...,32.0,V,25.0,V,20.0,V,14.0,V,8.0,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3876,28,79,60,8,28079060_8_8,2024,1,27,55.0,V,...,50.0,V,54.0,V,65.0,V,56.0,V,55.0,V
3877,28,79,60,8,28079060_8_8,2024,1,28,46.0,V,...,67.0,V,73.0,V,80.0,V,67.0,V,65.0,V
3878,28,79,60,8,28079060_8_8,2024,1,29,41.0,V,...,66.0,V,61.0,V,66.0,V,61.0,V,51.0,V
3879,28,79,60,8,28079060_8_8,2024,1,30,55.0,V,...,78.0,V,76.0,V,76.0,V,76.0,V,78.0,V


Nos vamos a quedar sólo con las estaciones que aparecen en las mediciones del mes de enero de 2024

In [7]:
estaciones = contaminacion_mes_madrid['ESTACION'].unique()

In [8]:
estaciones_df = estaciones_df[estaciones_df['CODIGO_CORTO'].isin(estaciones)]

## Representación en un mapa de las estaciones de medición

In [9]:
def dms_to_decimal(dms_str):
    """
    Convierte cualquier cosa como:
      "3º 40' 57''O", "3°40'57.00\"O", "40° 25' 25.98\"N", etc.
    a decimal (- para S/O).
    """
    s = dms_str.strip()
    hemi = s[-1].upper()
    dms_part = s[:-1]
    # Extrae todas las piezas numéricas (grados, minutos, segundos)
    nums = re.findall(r"[\d\.]+", dms_part)
    if len(nums) != 3:
        raise ValueError(f"Formato inesperado (esperaba 3 números): {dms_str!r}")
    deg, minute, sec = map(float, nums)
    dd = deg + minute/60 + sec/3600
    if hemi in ('S', 'O'):  # Sur y Oeste negativos
        dd = -dd
    return dd


In [10]:
estaciones_df['lon_dd'] = estaciones_df['LONGITUD_ETRS89'].apply(dms_to_decimal)
estaciones_df['lat_dd'] = estaciones_df['LATITUD_ETRS89'].apply(dms_to_decimal)

In [11]:
centro = [ estaciones_df['lat_dd'].mean(), estaciones_df['lon_dd'].mean() ]

m = folium.Map(location=centro, zoom_start=13)

# 2) Añadir marcadores
for _, row in estaciones_df.iterrows():
    folium.Marker(
        location=[row['lat_dd'], row['lon_dd']],
        popup=row['ESTACION']
    ).add_to(m)
display(m)

Como el objetivo de este trabajo es el estudio de la relación entre la intensidad del tráfico y la concentración de NO2. Vamos a eliminar del conjunto de datos las estaciones que podrían ser outliers, es decir las estaciones situadas dentro de parques (Casa de Campo, El Retiro, Juan Carlos I y  El Pardo) así como la estación de Barajas.

In [12]:
estaciones_eliminar = ['Barajas Pueblo', 'Casa de Campo', 'El Pardo', 'Parque del Retiro', 'Juan Carlos I']
estaciones_df = estaciones_df[~estaciones_df['ESTACION'].isin(estaciones_eliminar)]


## Preprocesamiento de datos

A continuación llevaremos a cabo el preprocesamiento de los datos de contaminación. Actualmente estamos trabajando con un mes de datos, por lo que crearemos una función para posteriormente poder reutilizar este procesamiento en el resto de meses.

Los datos originales tienen una frecuencia diaria, y cada fila representa las mediciones de un contaminante en una estación concreta a lo largo de un dia. Por tanto el procesamiento consistirá en:

* Filtrar para quedarnos sólo con las mediciones de las estaciones que nos interesan y del contaminante de estudio NO2
* Modificar la frecuencia para que pase a ser horaria en lugar de diaria

In [13]:
def procesar_serie_horaria(df, contaminante, estaciones):
    """
    Convierte un DataFrame mensual de contaminación (con columnas H01–H24 y V01–V24)
    en una serie horaria indexada por datetime, filtrando por magnitud y estaciones.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame que contiene, al menos, las columnas:
        ['MAGNITUD','ESTACION','ANO','MES','DIA','H01'…'H24','V01'…'V24']
    contaminante : int o str
        Valor de la columna MAGNITUD a filtrar (p.ej. 8).
    estaciones : list
        Lista de códigos de estación a conservar (valores de la columna ESTACION).

    Devuelve:
    ---------
    pd.DataFrame con índice datetime y columnas:
        ['ESTACION','medicion','flag']
    """
    # 1) Filtrar por magnitud y por estaciones
    df_f = df[
        (df['MAGNITUD'] == contaminante) &
        (df['ESTACION'].isin(estaciones))
    ].copy()
    if df_f.empty:
        return pd.DataFrame(
            columns=['ESTACION','medicion','flag'],
            index=pd.DatetimeIndex([], name='fecha_hora')
        )

    # 2) Desplegar H01–H24 y V01–V24 en formato “long”
    df_long = pd.wide_to_long(
        df_f,
        stubnames=['H','V'],
        i=['ANO','MES','DIA','ESTACION'],
        j='hora',
        sep='',             # porque las columnas son H01, H02, …
        suffix='\\d+'       # para capturar el número de hora
    ).reset_index()

    # 3) Renombrar columnas de valor y flag
    df_long = df_long.rename(columns={'H':'medicion','V':'flag'})

    # 4) Construir la columna datetime correctamente
    fecha_base = pd.to_datetime({
        'year':  df_long['ANO'],
        'month': df_long['MES'],
        'day':   df_long['DIA'],
    })
    # sumamos la 'hora' (1–24); si hora==24 → añade 24h → pasa al día siguiente 00:00
    df_long['fecha_hora'] = fecha_base + pd.to_timedelta(df_long['hora'], unit='h')

    # 5) Seleccionar y ordenar, poniendo fecha_hora como índice
    resultado = (
        df_long.loc[:, ['ESTACION','fecha_hora','medicion','flag']]
               .set_index('fecha_hora')
               .sort_index()
    )

    return resultado


In [14]:
contaminacion_mes_madrid = procesar_serie_horaria(contaminacion_mes_madrid,
                               contaminante,
                               estaciones)
contaminacion_mes_madrid.head(10)

,ESTACION,medicion,flag
fecha_hora,,,
2024-01-01 01:00:00,4,15.0,V
2024-01-01 01:00:00,58,12.0,V
2024-01-01 01:00:00,57,31.0,V
2024-01-01 01:00:00,8,22.0,V
2024-01-01 01:00:00,56,32.0,V
2024-01-01 01:00:00,55,38.0,V
2024-01-01 01:00:00,54,44.0,V
2024-01-01 01:00:00,50,28.0,V
2024-01-01 01:00:00,11,23.0,V


In [15]:
contaminacion_mes_madrid.rename(columns={'ESTACION':'CODIGO_CORTO'}, inplace=True)

# Cargando datos del tráfico del mes de Enero 2024

A continuación vamos a cargar los datos de tráfico medidos cada 15 minutos en diferentes estaciones de Madrid, y los metadatos asociados a estos detectores del tráfico

In [16]:
trafico_mes_madrid = pd.read_csv('https://www.dropbox.com/scl/fi/x05ygc88oay5ztaawwwvc/01-2024.csv?rlkey=8lfkm3j4lsa6aahwzss0hsvkb&st=upqh55v6&dl=1', sep=';')
trafico_mes_madrid.head(10)

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,2024-01-01 00:00:00,C30,300,1.0,0,57.0,N,5
1,1001,2024-01-01 00:15:00,C30,168,0.0,0,62.0,N,5
2,1001,2024-01-01 00:30:00,C30,540,2.0,0,61.0,N,5
3,1001,2024-01-01 00:45:00,C30,1140,3.0,0,64.0,N,5
4,1001,2024-01-01 01:00:00,C30,1920,5.0,0,62.0,N,5
5,1001,2024-01-01 01:15:00,C30,2196,5.0,0,59.0,N,5
6,1001,2024-01-01 01:30:00,C30,2436,7.0,0,61.0,N,5
7,1001,2024-01-01 01:45:00,C30,2892,9.0,0,60.0,N,5
8,1001,2024-01-01 02:00:00,C30,2280,6.0,0,59.0,N,5
9,1001,2024-01-01 02:15:00,C30,1944,7.0,0,57.0,N,5


In [17]:
estaciones_trafico_df = pd.read_csv('https://www.dropbox.com/scl/fi/wy21x3lhb5cajk95ktr9f/pmed_ubicacion_01-2024.csv?rlkey=vtyw0lssw2ltq5uvkdl0hmxzc&st=chw5lnte&dl=1', sep=';')
estaciones_trafico_df.head(10)

,tipo_elem,distrito,id,cod_cent,nombre,utm_x,utm_y,longitud,latitud
0,URB,4.0,3840,01001,Jose Ortega y Gasset E-O - Pº Castellana-Serrano,441615.343347,4.475768e+06,-3.688323,40.430502
1,URB,4.0,3841,01002,Jose Ortega y Gasset O-E - Serrano-Pº Castellana,441705.882340,4.475770e+06,-3.687256,40.430524
2,URB,1.0,3842,01003,Pº Recoletos N-S - Almirante-Prim,441319.371258,4.474841e+06,-3.691727,40.422132
3,URB,4.0,3843,01004,Pº Recoletos S-N - Pl. Cibeles- Recoletos,441301.632986,4.474764e+06,-3.691929,40.421433
4,URB,4.0,3844,01005,(AFOROS) Pº Castellana S-N - Eduardo Dato - P...,441605.765072,4.476132e+06,-3.688470,40.433782
5,URB,4.0,3845,01006,Pº Recoletos S-N - Villanueva-Jorge Juan,441382.968851,4.474994e+06,-3.690991,40.423512
6,URB,1.0,3846,01007,Sagasta 20 O-E - Mejia Lequerica-Pl. Alonso Ma...,440758.289091,4.475515e+06,-3.698403,40.428164
7,URB,7.0,3847,01008,Almagro 7 N-S - Zurbano-Pl. Alonso Martinez,441085.139018,4.475582e+06,-3.694556,40.428791
8,URB,7.0,3848,01009,(AFOROS) Genova 13 E-O - Zurbano-Campoamor,441114.888348,4.475375e+06,-3.694187,40.426925
9,URB,1.0,3849,01010,PL. SANTA BARBARA S-N(SERRANO ANGUITA-PL. ALO...,440913.426928,4.475372e+06,-3.696561,40.426881


## Asignación de estaciones de tráfico a las estaciones de contaminación

Lo siguiente que vamos a hacer es para cada punto de medición de NO2 en el aire se le asignarán una serie de puntos de medición de la intesidad del trafico. Para ello, para cada punto de medición de NO2 se definirá un radio de 200 metros, y todos los puntos de medición del trafico que estén dentro de este rango se asignarán a este punto de medición de calidad del aire. Si un punto de medición de calidad del aire dispone de menos de 3 puntos de medición de tráfico en un rango de 200 metros se descartará del análisis

In [18]:
estaciones_trafico = trafico_mes_madrid['id'].unique()
estaciones_trafico_df = estaciones_trafico_df[estaciones_trafico_df['id'].isin(estaciones_trafico)]
estaciones_trafico_df = estaciones_trafico_df[estaciones_trafico_df['tipo_elem'] == 'URB']

In [19]:
import geopandas as gpd
from shapely.geometry import Point

aire_gdf = gpd.GeoDataFrame(
    estaciones_df.copy(),
    geometry=gpd.points_from_xy(estaciones_df.lon_dd,
                                estaciones_df.lat_dd),
    crs="EPSG:4326"
)

traf_gdf = gpd.GeoDataFrame(
    estaciones_trafico_df.copy(),
    geometry=gpd.points_from_xy(estaciones_trafico_df.longitud,
                                estaciones_trafico_df.latitud),
    crs="EPSG:4326"
)

aire_gdf  = aire_gdf.to_crs("EPSG:25830")
traf_gdf  = traf_gdf.to_crs("EPSG:25830")

In [20]:
radio = 200
aire_gdf["buffer"] = aire_gdf.geometry.buffer(radio)


In [21]:
aire_buf = aire_gdf.set_geometry("buffer")

joined = gpd.sjoin(
    traf_gdf,
    aire_buf[["CODIGO_CORTO", "buffer"]],
    how="inner",
    predicate="within"
)


In [22]:
cnt = joined.groupby("CODIGO_CORTO").size()
stations_keep = cnt[cnt >= 3].index
joined = joined[joined.CODIGO_CORTO.isin(stations_keep)].copy()
aire_gdf = aire_gdf[aire_gdf.CODIGO_CORTO.isin(stations_keep)].copy()

In [23]:
est_geom = aire_gdf.reset_index().set_index("CODIGO_CORTO").geometry
joined["distancia_m"] = joined.apply(
    lambda r: r.geometry.distance(est_geom.loc[r.CODIGO_CORTO]), axis=1
).astype(float)
joined["peso"] = 1 / joined["distancia_m"].clip(lower=1)**2

asignaciones = (
    joined.loc[:, ["CODIGO_CORTO", "id", "distancia_m", "peso"]]
          .rename(columns={"id": "sensor_trafico_id"})
          .sort_values(["CODIGO_CORTO", "distancia_m"])
          .reset_index(drop=True)
)

In [24]:
asignaciones.groupby("CODIGO_CORTO").size().describe()


,0
count,10.000000
mean,7.000000
std,3.126944
min,3.000000
25%,5.000000
50%,6.500000
75%,8.000000
max,13.000000


A continuación graficaremos en un mapa cada punto de medición de calidad del aire y sus respectivos detectores de tráfico

In [25]:
import folium
import geopandas as gpd

aire_pts = aire_gdf.to_crs("EPSG:4326")
traf_sel = (
    gpd.GeoDataFrame(estaciones_trafico_df,
                     geometry=gpd.points_from_xy(estaciones_trafico_df.longitud,
                                                estaciones_trafico_df.latitud),
                     crs="EPSG:4326")
      .merge(asignaciones[["sensor_trafico_id", "CODIGO_CORTO"]],
             left_on="id", right_on="sensor_trafico_id",
             how="inner")
)


center = [aire_pts.geometry.y.mean(), aire_pts.geometry.x.mean()]
m = folium.Map(location=center, zoom_start=12, control_scale=True)

palette = [
    'red','blue','green','purple','orange','darkred','lightred','beige',
    'darkblue','darkgreen','cadetblue','darkpurple','white','pink',
    'lightblue','lightgreen','gray','black','lightgray','brown'
]

colors = {}

for i, row in aire_pts.iterrows():
    est = row.CODIGO_CORTO
    colors[est] = palette[i % len(palette)]
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=7,
        color=colors[est],
        fill=True, fill_opacity=0.9,
        popup=f"Estación aire {est}"
    ).add_to(m)




for _, row in traf_sel.iterrows():
    est = row.CODIGO_CORTO
    col = colors.get(est)
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color=col,
        fill=True, fill_opacity=0.4,
        popup=f"Sensor tráfico {int(row.id)} → Est. {est}"
    ).add_to(m)

    est_geom = aire_pts.loc[aire_pts.CODIGO_CORTO == est, "geometry"].iloc[0]
    folium.PolyLine(
        locations=[[row.geometry.y, row.geometry.x],
                   [est_geom.y, est_geom.x]],
        color=col, weight=1, opacity=0.5
    ).add_to(m)

m


Vamos a graficar en un mapa la relación entre la intensidad del tráfico en las inmediaciones de un sensor de calidad del aire y la concentración de NO₂. La intensidad del tráfico se ha calculado como la media ponderada inversa de la distancia entre el sensor de calidad del aire y los distintos sensores de tráfico cercanos.

En el gráfico resultante, cada círculo representa un sensor de calidad del aire. Cuanto mayor es el tamaño del círculo, mayor es la intensidad del tráfico en su entorno. Cuanto más oscuro es el color del círculo, mayor es la concentración de NO₂ registrada en ese punto.

In [26]:
import pandas as pd, geopandas as gpd, folium, branca.colormap as cm

for df_name in ["estaciones_df", "contaminacion_mes_madrid", "asignaciones"]:
    globals()[df_name]["CODIGO_CORTO"] = globals()[df_name]["CODIGO_CORTO"].astype("int64")
trafico_mes_madrid["id"]               = trafico_mes_madrid["id"].astype("int64")
asignaciones["sensor_trafico_id"]      = asignaciones["sensor_trafico_id"].astype("int64")

no2_mean = (
    contaminacion_mes_madrid
      .groupby("CODIGO_CORTO", as_index=False)["medicion"]
      .mean()
      .rename(columns={"medicion": "no2_mean"})
)

traf_tmp = (
    trafico_mes_madrid
      .merge(asignaciones[["sensor_trafico_id", "CODIGO_CORTO", "peso"]],
             left_on="id", right_on="sensor_trafico_id", how="inner")
)
traf_media = (
    traf_tmp
      .assign(Ip=lambda d: d.intensidad * d.peso)
      .groupby("CODIGO_CORTO", as_index=False)
      .agg(Ip_sum=("Ip","sum"), peso_sum=("peso","sum"))
      .assign(trafico_mean=lambda d: d.Ip_sum / d.peso_sum)
      .loc[:, ["CODIGO_CORTO", "trafico_mean"]]
)

aire_pts = gpd.GeoDataFrame(
    estaciones_df,
    geometry=gpd.points_from_xy(estaciones_df.lon_dd, estaciones_df.lat_dd),
    crs="EPSG:4326"
)

map_df = (
    aire_pts[["CODIGO_CORTO","geometry"]]
      .merge(no2_mean,  on="CODIGO_CORTO")
      .merge(traf_media, on="CODIGO_CORTO")
)
no2_min,  no2_max  = map_df.no2_mean.min(),   map_df.no2_mean.max()
traf_min, traf_max = map_df.trafico_mean.min(), map_df.trafico_mean.max()

colormap = cm.linear.YlOrRd_09.scale(no2_min, no2_max)
colormap.caption = "NO₂ medio (µg/m³)"

m = folium.Map(location=[map_df.geometry.y.mean(),
                         map_df.geometry.x.mean()],
               zoom_start=12, control_scale=True)

for _, row in map_df.iterrows():
    radius = 4 + 16 * (row.trafico_mean - traf_min) / (traf_max - traf_min)  # 4-20 px

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=radius,
        color=colormap(row.no2_mean),
        fill=True, fill_opacity=0.8,
        popup=(f"<b>Estación {row.CODIGO_CORTO}</b><br>"
               f"NO₂ medio: {row.no2_mean:.1f} µg/m³<br>"
               f"Tráfico medio (ponderado): {row.trafico_mean:.0f} veh/h")
    ).add_to(m)

colormap.add_to(m)
m

Se puede observar que los resultados no son los esperados ya que existen circulos con intensidad de tráfico elevada y que sin embargo, sus niveles de contaminación no son muy elevados. En cambio hay puntos con poca intensidad de tráfico con valores de concentración de NO2 muy elevados. Esto tiene sentido ya que la concentración puede depender de otros factores externos como la temperature o el viento en esa zona, el tipo de vehículos que pueden circular debido a restricciones legales etc.

# Transformación a una serie temporal

Para no influir en el resultado debido al uso de datos de otras estaciones, vamos a utilizar los datos sólo de la estación de calidad del aire con id 38 que es Cuatro Caminos.

In [27]:
asignaciones

,CODIGO_CORTO,sensor_trafico_id,distancia_m,peso
0,4,4284,52.165291,0.000367
1,4,4286,68.378359,0.000214
2,4,4285,101.840933,0.000096
3,4,4283,128.935621,0.000060
4,4,10647,148.439220,0.000045
...,...,...,...,...
65,56,11006,163.018840,0.000038
66,56,10613,164.614078,0.000037
67,56,5084,165.799129,0.000036
68,56,10579,176.703225,0.000032


In [30]:
estaciones_contaminacion_a_analizar = [38]
estaciones_trafico_a_analizar = asignaciones[asignaciones['CODIGO_CORTO'].isin(estaciones_contaminacion_a_analizar)]['sensor_trafico_id'].unique()


In [31]:
# -------- diagnostico_trafico.py -------------
print("Shape   :", trafico_mes_madrid.shape)

# 1. Muestra las primeras filas
print("\n--- HEAD ----------------")
print(trafico_mes_madrid.head())

# 2. Dtypes de las columnas clave (si existen)
for col in ["id", "fecha_hora", "intensidad", "ANO", "MES", "DIA", "HORA", "MIN"]:
    if col in trafico_mes_madrid.columns:
        print(f"{col:12s} →", trafico_mes_madrid[col].dtype)

# 3. Rango temporal
if "fecha_hora" in trafico_mes_madrid.columns:
    print("\nRango fecha_hora:",
          trafico_mes_madrid["fecha_hora"].min(),
          "→",
          trafico_mes_madrid["fecha_hora"].max())


Shape   : (13343452, 9)

--- HEAD ----------------
     id                fecha tipo_elem  intensidad  ocupacion  carga  vmed  \
0  1001  2024-01-01 00:00:00       C30         300        1.0      0  57.0   
1  1001  2024-01-01 00:15:00       C30         168        0.0      0  62.0   
2  1001  2024-01-01 00:30:00       C30         540        2.0      0  61.0   
3  1001  2024-01-01 00:45:00       C30        1140        3.0      0  64.0   
4  1001  2024-01-01 01:00:00       C30        1920        5.0      0  62.0   

  error  periodo_integracion  
0     N                    5  
1     N                    5  
2     N                    5  
3     N                    5  
4     N                    5  
id           → int64
intensidad   → int64


In [32]:
import pandas as pd
from functools import partial

def procesar_trafico_horario(df_traf,
                             asignaciones,
                             codigos_corto,
                             columna_fecha="fecha",
                             tz="Europe/Madrid",
                             freq="1H"):
    """
    A partir de la tabla de tráfico en cuartos de hora, produce
    la intensidad ponderada a frecuencia <freq> para las estaciones
    indicadas en codigos_corto.

    Parámetros
    ----------
    df_traf : pd.DataFrame
        Necesita columnas ['id', columna_fecha, 'intensidad'].
    asignaciones : pd.DataFrame
        Necesita ['sensor_trafico_id','CODIGO_CORTO','peso'].
    codigos_corto : list-like
        Lista de CODIGO_CORTO (int o str) de las estaciones de calidad de aire
        que quieres analizar (p.ej. [38] para Cuatro Caminos).
    columna_fecha : str
        Nombre de la columna con el timestamp. Por defecto 'fecha'.
    tz : str
        Zona horaria de trabajo (todos los datasets deben coincidir).
    freq : str
        Resolución de salida. '1H' produce serie horaria.

    Devuelve
    --------
    pd.DataFrame  -> index = fecha_hora ; columnas = ['CODIGO_CORTO','trafico_pond']
    """

    # 0 ▸ filtramos asignaciones a las estaciones deseadas
    asig_sel = asignaciones[asignaciones["CODIGO_CORTO"].isin(codigos_corto)]

    if asig_sel.empty:
        raise ValueError("No se encontraron sensores para esos CODIGO_CORTO")

    sensores = asig_sel["sensor_trafico_id"].unique()

    # 1 ▸ filtramos tráfico por sensores relevantes
    traf = df_traf[df_traf["id"].isin(sensores)].copy()

    # 2 ▸ timestamp a datetime con zona horaria
    traf[columna_fecha] = (
        pd.to_datetime(traf[columna_fecha])
          .dt.tz_localize(tz, nonexistent="shift_forward")
    )

    # 3 ▸ enlazamos peso + CODIGO_CORTO
    traf = traf.merge(asig_sel,
                      left_on="id",
                      right_on="sensor_trafico_id",
                      how="left")

    # 4 ▸ columna auxiliar Ip = intensidad * peso
    traf["Ip"] = traf["intensidad"].astype(float) * traf["peso"]

    # 5 ▸ Snap a la hora inferior (00, 01, …)
    traf["fecha_hora"] = traf[columna_fecha].dt.floor(freq)

    # 6 ▸ media ponderada hora-a-hora
    traf_h = (
        traf.groupby(["CODIGO_CORTO", "fecha_hora"])
            .apply(lambda g: g["Ip"].sum() / g["peso"].sum())
            .rename("trafico_pond")
            .reset_index()
            .set_index("fecha_hora")
            .sort_index()
    )

    return traf_h


In [33]:
trafico_horario = procesar_trafico_horario(
    df_traf          = trafico_mes_madrid,
    asignaciones     = asignaciones,
    codigos_corto    = estaciones_contaminacion_a_analizar
)

<ipython-input-32-59dba3edd31d>:63: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  traf["fecha_hora"] = traf[columna_fecha].dt.floor(freq)
<ipython-input-32-59dba3edd31d>:68: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g["Ip"].sum() / g["peso"].sum())


In [35]:
serie_merged = (
    contaminacion_mes_madrid.reset_index()          # fecha_hora pasa a columna
      .merge(trafico_horario.reset_index(),
             on=["CODIGO_CORTO", "fecha_hora"],
             how="inner")
      .set_index("fecha_hora")
      .sort_index()
)

ValueError: You are trying to merge on datetime64[ns] and datetime64[ns, Europe/Madrid] columns for key 'fecha_hora'. If you wish to proceed you should use pd.concat